In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.signal import savgol_filter
import warnings
warnings.filterwarnings('ignore')

# Load data
train_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
test_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
submission = pd.DataFrame()
submission['ID'] = test_df['ID']

# NDVI columns
ndvi_cols = [col for col in train_df.columns if col.endswith('_N')]
print(f"Found {len(ndvi_cols)} NDVI columns")

# Convert safely
train_df[ndvi_cols] = train_df[ndvi_cols].apply(pd.to_numeric, errors='coerce')
test_df[ndvi_cols] = test_df[ndvi_cols].apply(pd.to_numeric, errors='coerce')

# Fill NA - more efficient
for col in ndvi_cols:
    train_df[col] = train_df[col].fillna(train_df[col].median())
    test_df[col] = test_df[col].fillna(test_df[col].median())

# Optional: Smoothing (comment out if too slow)
def smooth_ndvi_fast(df):
    smoothed_df = df.copy()
    for col in ndvi_cols:
        if len(df) > 5:
            try:
                smoothed_df[col] = savgol_filter(df[col], min(5, len(df)//2*2+1), 2)
            except:
                pass
    return smoothed_df

print("Applying smoothing...")
train_df = smooth_ndvi_fast(train_df)
test_df = smooth_ndvi_fast(test_df)

# Enhanced Feature Engineering
def add_comprehensive_features(df):
    print("Adding comprehensive features...")
    
    # Basic statistical features
    df['ndvi_mean'] = df[ndvi_cols].mean(axis=1)
    df['ndvi_std'] = df[ndvi_cols].std(axis=1)
    df['ndvi_min'] = df[ndvi_cols].min(axis=1)
    df['ndvi_max'] = df[ndvi_cols].max(axis=1)
    df['ndvi_range'] = df['ndvi_max'] - df['ndvi_min']
    df['ndvi_median'] = df[ndvi_cols].median(axis=1)
    df['ndvi_skew'] = df[ndvi_cols].skew(axis=1)
    df['ndvi_kurtosis'] = df[ndvi_cols].kurtosis(axis=1)
    df['ndvi_q25'] = df[ndvi_cols].quantile(0.25, axis=1)
    df['ndvi_q75'] = df[ndvi_cols].quantile(0.75, axis=1)
    df['ndvi_iqr'] = df['ndvi_q75'] - df['ndvi_q25']
    
    # Trend analysis
    df['ndvi_trend'] = df[ndvi_cols].apply(lambda row: np.polyfit(range(len(row)), row, 1)[0], axis=1)
    
    # Seasonal patterns
    thirds = np.array_split(ndvi_cols, 3)
    df['early_mean'] = df[thirds[0]].mean(axis=1)
    df['mid_mean'] = df[thirds[1]].mean(axis=1)
    df['late_mean'] = df[thirds[2]].mean(axis=1)
    df['early_trend'] = df[thirds[0]].apply(lambda row: np.polyfit(range(len(row)), row, 1)[0], axis=1)
    df['mid_trend'] = df[thirds[1]].apply(lambda row: np.polyfit(range(len(row)), row, 1)[0], axis=1)
    df['late_trend'] = df[thirds[2]].apply(lambda row: np.polyfit(range(len(row)), row, 1)[0], axis=1)
    
    # Advanced seasonal features
    df['early_late_diff'] = df['early_mean'] - df['late_mean']
    df['peak_position'] = df[ndvi_cols].apply(lambda row: np.argmax(row) / len(row), axis=1)
    df['valley_position'] = df[ndvi_cols].apply(lambda row: np.argmin(row) / len(row), axis=1)
    df['peak_valley_diff'] = df['peak_position'] - df['valley_position']
    
    # Growth rate and acceleration
    def calculate_acceleration(row):
        if len(row) < 3:
            return 0
        diff1 = np.diff(row)
        diff2 = np.diff(diff1)
        return np.mean(diff2) if len(diff2) > 0 else 0
    
    df['ndvi_acceleration'] = df[ndvi_cols].apply(calculate_acceleration, axis=1)
    
    # Stability measures
    df['ndvi_cv'] = df['ndvi_std'] / (df['ndvi_mean'] + 1e-8)  # Coefficient of variation
    df['above_median_ratio'] = df[ndvi_cols].apply(lambda row: (row > row.median()).sum() / len(row), axis=1)
    
    # Interaction features
    df['trend_x_mean'] = df['ndvi_trend'] * df['ndvi_mean']
    df['range_x_skew'] = df['ndvi_range'] * df['ndvi_skew']
    df['std_x_mean'] = df['ndvi_std'] * df['ndvi_mean']
    df['peak_pos_x_max'] = df['peak_position'] * df['ndvi_max']
    
    # Log transformations for skewed features
    df['log_std'] = np.log1p(df['ndvi_std'])
    df['log_range'] = np.log1p(df['ndvi_range'])
    df['log_iqr'] = np.log1p(df['ndvi_iqr'])
    
    # Percentile-based features
    df['ndvi_p10'] = df[ndvi_cols].quantile(0.1, axis=1)
    df['ndvi_p90'] = df[ndvi_cols].quantile(0.9, axis=1)
    df['extreme_range'] = df['ndvi_p90'] - df['ndvi_p10']
    
    return df

train_df = add_comprehensive_features(train_df)
test_df = add_comprehensive_features(test_df)

# Comprehensive feature list
features = [
    # Basic statistics
    'ndvi_mean', 'ndvi_std', 'ndvi_min', 'ndvi_max', 'ndvi_range',
    'ndvi_median', 'ndvi_skew', 'ndvi_kurtosis', 'ndvi_q25', 'ndvi_q75', 'ndvi_iqr',
    
    # Trends
    'ndvi_trend', 'early_trend', 'mid_trend', 'late_trend',
    
    # Seasonal
    'early_mean', 'mid_mean', 'late_mean', 'early_late_diff',
    'peak_position', 'valley_position', 'peak_valley_diff',
    
    # Advanced
    'ndvi_acceleration', 'ndvi_cv', 'above_median_ratio',
    
    # Interactions
    'trend_x_mean', 'range_x_skew', 'std_x_mean', 'peak_pos_x_max',
    
    # Transformations
    'log_std', 'log_range', 'log_iqr',
    
    # Percentiles
    'ndvi_p10', 'ndvi_p90', 'extreme_range'
]

print(f"Total features created: {len(features)}")

X = train_df[features]
y = train_df['class']
X_test = test_df[features]

# Handle any infinite or NaN values
X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

# Encode target
label_map = {label: i for i, label in enumerate(y.unique())}
inverse_label_map = {i: label for label, i in label_map.items()}
y_encoded = y.map(label_map)

print(f"Target classes: {list(label_map.keys())}")

# Feature Selection - Select best features
selector = SelectKBest(score_func=f_classif, k=min(25, len(features)))
X_selected = selector.fit_transform(X, y_encoded)
X_test_selected = selector.transform(X_test)

selected_features = [features[i] for i in selector.get_support(indices=True)]
print(f"Selected {len(selected_features)} best features")

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Cross-validation setup
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=100)

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)

# 1. Test Logistic Regression with different max_iter
print("\n1. Testing Logistic Regression convergence:")
print("-" * 40)

iter_values = [500, 1000, 2000]
best_lr_score = 0
best_lr_iter = 500

for max_iter in iter_values:
    logreg = LogisticRegression(
        max_iter=max_iter, 
        solver='lbfgs', 
        multi_class='multinomial', 
        random_state=100
    )
    
    scores = cross_val_score(logreg, X_scaled, y_encoded, cv=cv, scoring='accuracy')
    score_mean = scores.mean()
    
    print(f"max_iter={max_iter:4d}: CV Accuracy = {score_mean:.4f} ± {scores.std():.4f}")
    
    if score_mean > best_lr_score:
        best_lr_score = score_mean
        best_lr_iter = max_iter

print(f"Best Logistic Regression: max_iter={best_lr_iter}, score={best_lr_score:.4f}")

# 2. Test Random Forest
print("\n2. Testing Random Forest:")
print("-" * 40)

rf = RandomForestClassifier(n_estimators=100, random_state=100, n_jobs=-1)
rf_scores = cross_val_score(rf, X_scaled, y_encoded, cv=cv, scoring='accuracy')
rf_score_mean = rf_scores.mean()

print(f"Random Forest: CV Accuracy = {rf_score_mean:.4f} ± {rf_scores.std():.4f}")

# 3. Hyperparameter tuning for best model
print("\n3. Hyperparameter Tuning:")
print("-" * 40)

if rf_score_mean > best_lr_score:
    print("Random Forest performed better - tuning RF parameters...")
    
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5]
    }
    
    grid_search = GridSearchCV(
        RandomForestClassifier(random_state=100, n_jobs=-1),
        param_grid,
        cv=3,
        scoring='accuracy',
        n_jobs=-1
    )
    
    grid_search.fit(X_scaled, y_encoded)
    best_model = grid_search.best_estimator_
    best_score = grid_search.best_score_
    
    print(f"Best RF params: {grid_search.best_params_}")
    print(f"Best RF score: {best_score:.4f}")
    
else:
    print("Logistic Regression performed better - tuning LR parameters...")
    
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'max_iter': [best_lr_iter]
    }
    
    grid_search = GridSearchCV(
        LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=100),
        param_grid,
        cv=3,
        scoring='accuracy'
    )
    
    grid_search.fit(X_scaled, y_encoded)
    best_model = grid_search.best_estimator_
    best_score = grid_search.best_score_
    
    print(f"Best LR params: {grid_search.best_params_}")
    print(f"Best LR score: {best_score:.4f}")

# Final training and prediction
print("\n" + "="*60)
print("FINAL PREDICTION")
print("="*60)

print(f"Training final model with CV score: {best_score:.4f}")
best_model.fit(X_scaled, y_encoded)

# Make predictions
y_pred_test = best_model.predict(X_test_scaled)
submission['class'] = [inverse_label_map[i] for i in y_pred_test]

# Feature importance (if available)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': selected_features,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10).to_string(index=False))

elif hasattr(best_model, 'coef_'):
    # For logistic regression, show features with highest absolute coefficients
    if len(best_model.coef_.shape) > 1:
        # Multi-class: take mean of absolute coefficients across classes
        coef_importance = np.mean(np.abs(best_model.coef_), axis=0)
    else:
        coef_importance = np.abs(best_model.coef_[0])
    
    feature_importance = pd.DataFrame({
        'feature': selected_features,
        'importance': coef_importance
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 Most Important Features (by coefficient magnitude):")
    print(feature_importance.head(10).to_string(index=False))

# Save submission
submission.to_csv("submission.csv", index=False)
print(f"\nSaved submission.csv with {len(submission)} predictions")
print("Model training completed successfully!")

Found 27 NDVI columns
Applying smoothing...
Adding comprehensive features...
Adding comprehensive features...
Total features created: 35
Target classes: ['water', 'forest', 'impervious', 'farm', 'grass', 'orchard']
Selected 25 best features

MODEL COMPARISON

1. Testing Logistic Regression convergence:
----------------------------------------
max_iter= 500: CV Accuracy = 0.8691 ± 0.0049
max_iter=1000: CV Accuracy = 0.8691 ± 0.0049
max_iter=2000: CV Accuracy = 0.8691 ± 0.0049
Best Logistic Regression: max_iter=500, score=0.8691

2. Testing Random Forest:
----------------------------------------
Random Forest: CV Accuracy = 0.8800 ± 0.0064

3. Hyperparameter Tuning:
----------------------------------------
Random Forest performed better - tuning RF parameters...
Best RF params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Best RF score: 0.8580

FINAL PREDICTION
Training final model with CV score: 0.8580

Top 10 Most Important Features:
      feature  importance
    lo

In [2]:
print(test_df.columns.tolist())


['Unnamed: 0', 'ID', '20150720_N', '20150602_N', '20150517_N', '20150501_N', '20150415_N', '20150330_N', '20150314_N', '20150226_N', '20150210_N', '20150125_N', '20150109_N', '20141117_N', '20141101_N', '20141016_N', '20140930_N', '20140813_N', '20140626_N', '20140610_N', '20140525_N', '20140509_N', '20140423_N', '20140407_N', '20140322_N', '20140218_N', '20140202_N', '20140117_N', '20140101_N', 'ndvi_mean', 'ndvi_std', 'ndvi_min', 'ndvi_max', 'ndvi_range', 'ndvi_median', 'ndvi_skew', 'ndvi_kurtosis', 'ndvi_q25', 'ndvi_q75', 'ndvi_iqr', 'ndvi_trend', 'early_mean', 'mid_mean', 'late_mean', 'early_trend', 'mid_trend', 'late_trend', 'early_late_diff', 'peak_position', 'valley_position', 'peak_valley_diff', 'ndvi_acceleration', 'ndvi_cv', 'above_median_ratio', 'trend_x_mean', 'range_x_skew', 'std_x_mean', 'peak_pos_x_max', 'log_std', 'log_range', 'log_iqr', 'ndvi_p10', 'ndvi_p90', 'extreme_range']
